<a href="https://colab.research.google.com/github/GasKitela/ia-utn/blob/main/Entrega2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Subida del zip

In [ ]:
from google.colab import files
import zipfile
import io


uploaded = files.upload()

data = zipfile.ZipFile(io.BytesIO(uploaded['Evoluciones_Agumon.zip']), 'r')
data.extractall()


Saving Evoluciones_Agumon.zip to Evoluciones_Agumon.zip


Lectura de los datos, entrenamiento del modelo y prediccion

In [25]:
from google.colab import files
import zipfile
import io
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import keras.optimizers as Optimizer


try: 
  %tensorflow_version 2.x
except Exception: 
  pass

ruta_agumon="/content/Evoluciones_Agumon/Agumon"
ruta_greymon="/content/Evoluciones_Agumon/Greymon"
ruta_metalgreymon="/content/Evoluciones_Agumon/Metalgreymon"
ruta_wargreymon="/content/Evoluciones_Agumon/Wargreymon"

agumon=[]
greymon=[]
metalgreymon=[]
wargreymon=[]
img_size=150

for img in os.listdir(ruta_agumon):
  if('jpg' in img):
    img = cv2.imread(os.path.join(ruta_agumon, img))
    img_resize = cv2.resize(img, (img_size,img_size), interpolation=cv2.INTER_AREA)
    agumon.append(img_resize)

print("Elementos agumon", len(agumon))

for img in os.listdir(ruta_greymon):
  if('jpg' in img):
    img = cv2.imread(os.path.join(ruta_greymon, img))
    img_resize = cv2.resize(img, (img_size,img_size), interpolation=cv2.INTER_AREA)
    greymon.append(img_resize)

print("Elementos greymon", len(greymon))

for img in os.listdir(ruta_metalgreymon):
  if('jpg' in img):
    img = cv2.imread(os.path.join(ruta_metalgreymon, img))
    img_resize = cv2.resize(img, (img_size,img_size), interpolation=cv2.INTER_AREA)
    metalgreymon.append(img_resize)

print("Elementos metalgreymon", len(metalgreymon))

for img in os.listdir(ruta_wargreymon):
  if('jpg' in img):
    img = cv2.imread(os.path.join(ruta_wargreymon, img))
    img_resize = cv2.resize(img, (img_size,img_size), interpolation=cv2.INTER_AREA)
    wargreymon.append(img_resize)

print("Elementos wargreymon", len(wargreymon))

Agumon = np.array(agumon)
Greymon = np.array(greymon)
Metalgreymon = np.array(metalgreymon)
Wargreymon = np.array(wargreymon)

# Juntar imágenes
imagenes2 = np.concatenate([Agumon, Greymon])
imagenes1 = np.concatenate([imagenes2, Metalgreymon])
imagenes = np.concatenate([imagenes1, Wargreymon])

Imagenes = np.array(imagenes)

etiquetas_agumon = np.repeat(0, 21)
etiquetas_greymon = np.repeat(1, 26)
etiquetas_metalgreymon = np.repeat(2, 21)
etiquetas_wargreymon = np.repeat(3, 26)

labels2 = np.concatenate([etiquetas_agumon, etiquetas_greymon])
labels1 = np.concatenate([labels2, etiquetas_metalgreymon])
labels = np.concatenate([labels1, etiquetas_wargreymon])

Labels = np.array(labels)

clases = ['Agumon', 'Greymon', 'Metalgreymon', 'Wargreymon']

plt.figure(figsize=(10, 10))

for i in range(94):
  plt.subplot(10, 10, i + 1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  img = Imagenes[i]
  plt.imshow(img)
  plt.xlabel(clases[Labels[i]])

plt.show()

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(150, 150, 3)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(Imagenes, Labels, test_size=0.2, random_state=42)

trained = model.fit(Imagenes, Labels, epochs=92)

######################################################################################

# Obtener las predicciones en el conjunto de prueba
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print("y_pred_classes: ", y_pred_classes)

# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred_classes)

# Calcular la exactitud
accuracy = accuracy_score(y_test, y_pred_classes)

# Calcular la precisión, recuperación y otras métricas
report = classification_report(y_test, y_pred_classes, target_names=clases)

# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(cm)

# Mostrar la exactitud
print("Exactitud:", accuracy)

# Mostrar las métricas
print("Métricas de clasificación:")
print(report)

######################################################################################

continuar = True

while continuar:
  print("Cargar imagen prediccion.jpg para clasificar:")
  uploaded = files.upload()

  # Obtener el nombre del archivo subido
  nombre_archivo = next(iter(uploaded))
  
  # Leer la imagen usando el nombre del archivo subido
  img = cv2.imdecode(np.frombuffer(uploaded[nombre_archivo], np.uint8), cv2.IMREAD_COLOR)

  # Mostrar información de la imagen cargada
  print(f"Se ha cargado la imagen: {nombre_archivo}")

  #img = cv2.imread("/content/prediccion.jpg")
  img_resize = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_AREA)
  imagen_pred = np.expand_dims(img_resize, 0)

  prediction = model.predict(imagen_pred)

  threshold = 0.5  # Umbral mínimo de probabilidad

  max_probability = np.max(prediction)
  predicted_class = np.argmax(prediction)

  if max_probability < threshold:
    print("La imagen no se ajusta a ninguna clasificación.")
  else:
    print("La clasificación predecida es", clases[predicted_class])

  respuesta_usuario = getpass.getpass('Predecir otra vez? (s/n): ')


  if respuesta_usuario.lower() == "s":
    continuar = True
  else:
    continuar = False  # Salir del bucle si el usuario responde 'n'

print("Fin.")


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Elementos agumon 21
Elementos greymon 26


KeyboardInterrupt: ignored